In [1]:
import numpy
import scipy.io.wavfile
from scipy.fftpack import dct


#Step 1 loading a file

# Step 1

* First step is to load the wav file
* we can cut the signal by using a sample_rate - number of samples per second
  * 3.5 * sample_rate takes only first 3.5 seconds of the record

In [2]:
sample_rate, signal = scipy.io.wavfile.read('an4_dataset/train/an1-mblw-b.wav')  # File assumed to be in the same directory
signal = signal[0:int(3.5 * sample_rate)]  # Keep the first 3.5 seconds

# Step 2 - optional

* pre-emphasis
 1. enhances higher frequencies
 2. avoids numerical problems during FFT
 3. may improve SNR - Signal-to-noise ratio
 
* not required due to modern implementations of FFT have improved

In [3]:
pre_emphasis = 0.97
emphasized_signal = numpy.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])

# Step 3

* framing
 * signals are stable for a very short time
 * we therefore split signal into small frames to extract frequencies for FFT in these shorter relatively stable periods
 * allows for better approximation of frequencies
 
* typically 20 - 40 ms sequencies with approx. 10% overlapping
* stride of 0.02 is 20 ms and frame size of 40 ms is 0.04
 * frame_stride = 0.02
 * frame_size = 0.04

In [4]:
frame_size, frame_stride = 0.02, 0.01
frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate  # Convert from seconds to samples
signal_length = len(emphasized_signal)
frame_length = int(round(frame_length))
frame_step = int(round(frame_step))
num_frames = int(numpy.ceil(float(numpy.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame

pad_signal_length = num_frames * frame_step + frame_length
z = numpy.zeros((pad_signal_length - signal_length))
pad_signal = numpy.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal

indices = numpy.tile(numpy.arange(0, frame_length), (num_frames, 1)) + numpy.tile(numpy.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
frames = pad_signal[indices.astype(numpy.int32, copy=False)]

# Step 4

* Window function
  * Hamming window or another
  * counters FFT assumptions such as the data is infinite

In [5]:
frames *= numpy.hamming(frame_length)
# frames *= 0.54 - 0.46 * numpy.cos((2 * numpy.pi * n) / (frame_length - 1))  # Explicit Implementation 

# Step 5

* Fourier transform
  * short time fourier transform - STFT
  * on each step calculates frequency spectrum

* power spectrum
  * periodogram

In [6]:
NFFT = 512
mag_frames = numpy.absolute(numpy.fft.rfft(frames, NFFT))  # Magnitude of the FFT
pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # Power Spectrum

/usr/local/lib/python3.6/site-packages/mkl_fft/_numpy_fft.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfft_numpy(a, n=n, axis=axis)


# Step 6

* Filter banks
  * applies filter banks to the sound
  * uses Mel-scale
  * extracts spectrum to frequency bands
  * mimics human ear - more discriminative at lower frequencies and less discriminative at higher frequencies
* filter banks are triangular
* we always use number of filters - nfilt, typically around 40
  * nfilt = 40

In [7]:
nfilt = 40
low_freq_mel = 0
high_freq_mel = (2595 * numpy.log10(1 + (sample_rate / 2) / 700))  # Convert Hz to Mel
mel_points = numpy.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
hz_points = (700 * (10**(mel_points / 2595) - 1))  # Convert Mel to Hz
bin = numpy.floor((NFFT + 1) * hz_points / sample_rate)

fbank = numpy.zeros((nfilt, int(numpy.floor(NFFT / 2 + 1))))
for m in range(1, nfilt + 1):
    f_m_minus = int(bin[m - 1])   # left
    f_m = int(bin[m])             # center
    f_m_plus = int(bin[m + 1])    # right

    for k in range(f_m_minus, f_m):
        fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
    for k in range(f_m, f_m_plus):
        fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
filter_banks = numpy.dot(pow_frames, fbank.T)
filter_banks = numpy.where(filter_banks == 0, numpy.finfo(float).eps, filter_banks)  # Numerical Stability
filter_banks = 20 * numpy.log10(filter_banks)  # dB

# Step 7

* MFCC
  * Mel-frequency cepstral coefficients
  * mel filtering are highly correlated which can cause problems for ML algorithms
  * Applying DCT - Discrete cosine transform for decorrelation
  * Automatic speech recognition (ASR) uses coefficients between 2 - 13 which are retained
* Discarding the coefficients is because they represent fast changes in the filter bank coeffcients and do not contribute to ASR

In [8]:
num_ceps = 12
mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, 1 : (num_ceps + 1)] # Keep 2-13

* Also possible to apply sinusoidal liftering to de-emphasize higher MFCC which improves speech recognition in noisy signals

# Step 8

* mean normalization
* improves signal-to-noise

In [9]:
filter_banks -= (numpy.mean(filter_banks, axis=0) + 1e-8)
mfcc -= (numpy.mean(mfcc, axis=0) + 1e-8)

mfcc.shape

In [10]:
mfcc.shape

(148, 12)

In [11]:
print(mfcc.shape)

(148, 12)


In [12]:
mfcc

array([[-41.93547301,  12.11955534,  -3.74744433, ...,  29.307929  ,
         -4.05431875,   9.34707281],
       [-31.89631311,   4.16131126, -16.19114323, ...,  10.99546045,
        -20.04580869,   9.78383616],
       [-35.8446343 ,  16.94022011,  -2.95354261, ...,  26.50893761,
         -7.67204468,   5.48191619],
       ...,
       [ 17.12015833, -58.43328798, -22.92331547, ...,  -6.01145675,
        -13.8242886 ,  23.71481512],
       [ 37.034746  , -41.68287765, -36.86538942, ..., -12.53526094,
        -24.59991047,   9.57245889],
       [ 15.06486697, -36.19997078, -46.8258387 , ..., -20.42293341,
         -8.76024341,   7.08196808]])